In [1]:
%cd ..

/home/zarizky/projects/neural-autoregressive-object-co-occurrence


In [9]:
import json
from functools import partial

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pygraphviz as pgv
from IPython.display import Image
from tqdm.contrib.concurrent import process_map
from einet_addons.forest import gather_nodes

from utils import bin_g, pgmpy

In [3]:
df_train = pd.read_csv("dataset/coco2017-cooccurences-train.csv").iloc[:, 8:]
df_train.columns = [
    col.split("-")[0].replace("[", "").replace("]", "").upper()
    for col in df_train.columns
]

df_valid = pd.read_csv("dataset/coco2017-cooccurences-valid.csv").iloc[:, 8:]
df_valid.columns = df_train.columns

In [4]:
graph, model = bin_g.load("outputs/backup/coco2017_bin_20-Jun-2023.mat", df_train, " ")
roots = [node for node in graph.nodes if len(list(graph.predecessors(node))) <= 0]
forest = [gather_nodes(graph, root) for root in roots]

In [121]:
graph_cpy = nx.DiGraph()
graph_cpy.add_nodes_from(graph.nodes)
graph_cpy.add_edges_from(graph.edges)
dag_cpy = nx.relabel_nodes(
    graph_cpy, {k: k.replace(" ", "\n") for k in df_train.columns}
)

pgv = nx.nx_agraph.to_agraph(dag_cpy)

pgv.graph_attr['start'] = 0


independent_nodes = []
for i, tree in enumerate(forest, 1):
    #print([node for node in tree if node in df_train.columns])
    if len(tree) > 1:
        pgv.add_subgraph(tree, name=f't{i}', rankdir='TB')
    else:
        independent_nodes.extend(tree)
pgv_idp = pgv.add_subgraph(independent_nodes, name=f't{i+1}', rank='sink')

pgv.graph_attr['clusterrank'] = 'local'
#pgv.graph_attr["ratio"] = "0.5"
pgv.graph_attr["dpi"] = 512
pgv.graph_attr["overlap"] = "false"
pgv.graph_attr["margin"] = "0"
pgv.node_attr["shape"] = "box"
pgv.draw("outputs/bin_g_tree/graph.png", prog="dot")
pgv.draw("outputs/bin_g_tree/graph.pdf", prog="dot")

display(Image("outputs/bin_g_tree/graph.png"))

In [6]:
model.check_model()

for cpd in model.get_cpds():
    if np.any(cpd.values == 0):
        print(cpd)

total_params = 0
for cpd in model.get_cpds():
    total_params += cpd.values.size
total_params

4277

In [7]:
batch_size = 128
batch_train = [
    df_train.iloc[i : i + batch_size] for i in range(0, len(df_train), batch_size)
]
batch_valid = [
    df_valid.iloc[i : i + batch_size] for i in range(0, len(df_valid), batch_size)
]

log_posterior = partial(pgmpy.log_posterior, model=model)

total_train_ll = bin_g.log_likelihood(df_train, model)
avg_train_ll = total_train_ll / len(df_train)
train_bic = total_train_ll - total_params / 2 * np.log(len(df_train))

total_valid_ll = bin_g.log_likelihood(df_valid, model)
avg_valid_ll = total_valid_ll / len(df_valid)

total_train_lp = sum(process_map(log_posterior, batch_train))
avg_train_lp = total_train_lp / len(df_train)

total_valid_lp = sum(process_map(log_posterior, batch_valid))
avg_valid_lp = total_valid_lp / len(df_valid)

print(f"Avg. Train NLL: {-avg_train_ll:.4f}")
print(f"Avg. Valid NLL: {-avg_valid_ll:.4f}")
print(f"Avg. Train NLP: {-avg_train_lp:.4f}")
print(f"Avg. Valid NLP: {-avg_valid_lp:.4f}")
print(f"BIC: {train_bic:,.4f}")

  0%|          | 0/925 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Avg. Train NLL: 13.6124
Avg. Valid NLL: 13.7443
Avg. Train NLP: 12.8371
Avg. Valid NLP: 12.9711
BIC: -1,635,155.1840


In [8]:
with open("outputs/overall/bin_g.json", "w", encoding="utf-8") as file:
    json.dump(
        dict(
            model="BIN-G",
            kind="Structured Model",
            total_parameters=total_params,
            avg_train_nll=-avg_train_ll,
            avg_test_nll=-avg_valid_ll,
            avg_train_nlp=-avg_train_lp,
            avg_test_nlp=-avg_valid_lp,
            neg_bic=-train_bic,
        ),
        file,
        ensure_ascii=False,
        indent=4,
    )